In [1]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [2]:
data = np.load('CIFAR10_resnet50-keras_features.npz')

In [3]:
x_trn = np.vstack((data['features_training'], data['features_testing']))
y_trn = np.hstack((data['labels_training'], data['labels_testing']))

In [4]:
from sklearn.utils import shuffle
x_trn, y_trn = shuffle(x_trn, y_trn, random_state=0)

In [5]:
x_trn = np.reshape(x_trn, (-1, 2048))
x_tst, y_tst = x_trn[:100], y_trn[:100]
x_val, y_val = x_trn[100:1100], y_trn[100:1100]
x_trn, y_trn = x_trn[1100:], y_trn[1100:]

In [7]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [8]:
neigh.score(x_val, y_val)

0.852

In [9]:
neigh.score(x_tst, y_tst)

0.81

In [6]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial').fit(x_trn, y_trn)

/home/wbx/tensorflow/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [7]:
clf.score(x_val, y_val)

0.91

In [8]:
clf.score(x_tst, y_tst)

0.87

In [9]:
clf.score(x_trn, y_trn)

0.9502886247877759

In [7]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, input_dim=2048))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_trn, y_trn, epochs=5, batch_size=32)

Using TensorFlow backend.


Epoch 1/5
58900/58900 [==============================] - 11s 184us/step - loss: 0.4147 - acc: 0.8590
Epoch 2/5
58900/58900 [==============================] - 7s 113us/step - loss: 0.2936 - acc: 0.8993
Epoch 3/5
58900/58900 [==============================] - 7s 112us/step - loss: 0.2668 - acc: 0.9075
Epoch 4/5
58900/58900 [==============================] - 7s 114us/step - loss: 0.2493 - acc: 0.9139
Epoch 5/5
58900/58900 [==============================] - 7s 112us/step - loss: 0.2363 - acc: 0.9176


In [8]:
model.evaluate(x_tst, y_tst, batch_size=128)

100/100 [==============================] - 0s 505us/step


[0.39001956582069397, 0.8700000047683716]

In [9]:
model.evaluate(x_val, y_val, batch_size=128)

1000/1000 [==============================] - 0s 35us/step


[0.26140422391891477, 0.911]

In [10]:
model.fit(x_trn, y_trn, epochs=5, batch_size=32)

Epoch 1/5
58900/58900 [==============================] - 7s 115us/step - loss: 0.2279 - acc: 0.9210
Epoch 2/5
58900/58900 [==============================] - 6s 109us/step - loss: 0.2187 - acc: 0.9253
Epoch 3/5
58900/58900 [==============================] - 7s 121us/step - loss: 0.2126 - acc: 0.9269
Epoch 4/5
58900/58900 [==============================] - 7s 113us/step - loss: 0.2066 - acc: 0.9289
Epoch 5/5
58900/58900 [==============================] - 6s 110us/step - loss: 0.2015 - acc: 0.9309


In [11]:
model.evaluate(x_tst, y_tst, batch_size=128)

100/100 [==============================] - 0s 47us/step


[0.36326342821121216, 0.8700000047683716]

In [12]:
model.evaluate(x_val, y_val, batch_size=128)

1000/1000 [==============================] - 0s 30us/step


[0.2430015230178833, 0.9180000023841858]

In [13]:
model.fit(x_trn, y_trn, epochs=5, batch_size=32)

Epoch 1/5
58900/58900 [==============================] - 7s 117us/step - loss: 0.1968 - acc: 0.9324
Epoch 2/5
58900/58900 [==============================] - 7s 112us/step - loss: 0.1929 - acc: 0.9346
Epoch 3/5
58900/58900 [==============================] - 6s 96us/step - loss: 0.1887 - acc: 0.9349
Epoch 4/5
58900/58900 [==============================] - 6s 96us/step - loss: 0.1857 - acc: 0.9359
Epoch 5/5
58900/58900 [==============================] - 7s 113us/step - loss: 0.1823 - acc: 0.9382


In [14]:
model.evaluate(x_tst, y_tst, batch_size=128)

100/100 [==============================] - 0s 210us/step


[0.34444886445999146, 0.8600000143051147]

In [16]:
model.evaluate(x_val, y_val, batch_size=128)

1000/1000 [==============================] - 0s 30us/step


[0.25385982036590576, 0.9090000023841858]